<a href="https://colab.research.google.com/github/not-part-of-solution/Starving_cats/blob/main/operating_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Обучаем модель

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping

# Параметры для обработки изображений
image_size = (224, 224)
batch_size = 32
epochs = 20

# Генераторы данных для обучения и валидации
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,
    width_shift_range=0.4,
    height_shift_range=0.4,
    zoom_range=0.3,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Генератор для обучающих данных
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/operating_system/learning/',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    seed=42
)

# Генератор для валидационных данных
val_generator = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/operating_system/control/',
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    seed=42
)

# Создание модели
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Замораживание базовой модели

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(train_generator.num_classes, activation='softmax'))

# Компиляция модели
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

# Коллбек для досрочной остановки
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# Обучение модели
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Оттаивание базовой модели и повторное обучение
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
             loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Сохранение модели
model.save('/content/drive/MyDrive/operating_system/cat_recognition_model.h5')

Found 380 images belonging to 3 classes.
Found 219 images belonging to 3 classes.
Epoch 1/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 43s 3s/step - accuracy: 0.1724 - loss: 1.7129 - val_accuracy: 0.1370 - val_loss: 2.1413
Epoch 2/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 44s 4s/step - accuracy: 0.2181 - loss: 1.6087 - val_accuracy: 0.1324 - val_loss: 2.0417
Epoch 3/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.2695 - loss: 1.5200 - val_accuracy: 0.1324 - val_loss: 1.9539
Epoch 4/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.2312 - loss: 1.4880 - val_accuracy: 0.1324 - val_loss: 1.8702
Epoch 5/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.2433 - loss: 1.3819 - val_accuracy: 0.1416 - val_loss: 1.7901
Epoch 6/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 33s 3s/step - accuracy: 0.3199 - loss: 1.3067 - val_accuracy: 0.1689 - val_loss: 1.7118
Epoch 7/10
12/12 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.2685 - loss: 1.3853 - val_accuracy: 0.1872 - val_loss: 1.6354
Epoch 8/10
12/12 ━━━━━━━━━━━━━━━━━━━

# Тест модели

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2


def preprocess_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Не удалось открыть или найти изображение: {image_path}")

    img = cv2.resize(img, (224, 224))


    img = img.astype('float32') / 255.0

    img = np.expand_dims(img, axis=0)
    return img


# Загрузка обученной модели
model = tf.keras.models.load_model('/content/drive/MyDrive/operating_system/cat_recognition_model.h5')

# Предобработка нового изображения
image_path = '/content/drive/MyDrive/operating_system/19_45.jpg'
preprocessed_image = preprocess_image(image_path)

# Прогнозирование
predictions = model.predict(preprocessed_image)
predicted_class = np.argmax(predictions[0])
# Отображение результата
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}  # Разворачиваем словарь
predicted_label = class_labels[predicted_class]

print(f'Предсказанная метка: {predicted_label}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Предсказанная метка: Ева
